In [11]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm_notebook
import warnings
warnings.filterwarnings(action='ignore') #ignore all warnings include SSL warning for reqest.get

/Users/alanhou/miniforge3/envs/deadLinkFinder/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
#print error messages of every dead link under a given page
def printError(url):
    errorFound = False
    try:
        req = requests.get(url, timeout = 5, headers={'User-Agent': ''})
        req.raise_for_status()
    except Exception:
        return
    soup = BeautifulSoup(req.content, 'html.parser')
    content = soup.find_all("div", {"class": "region-content"})[0]
    for link in content.find_all('a'):
        linkUrl = link.get('href')
        text = link.get_text()
        if linkUrl == None:
            continue
        if "http" not in linkUrl:
            if "mailto:" or "tel:" in linkUrl:
                continue
            linkUrl = website + linkUrl
        try:
            r = requests.get(linkUrl, timeout = 5, headers={'User-Agent': ''}, verify=False)
            r.raise_for_status()
        except Exception:
            if not errorFound:
                print(url)
                errorFound = True
            print('\t'+text)
            print('\t'+linkUrl)
            print()
            # print('\t'+str(e))
    if errorFound:
        print()

In [13]:
website = "https://studentlife.umich.edu/parents"
req = requests.get(website+"/sitemap.xml", timeout = 5, headers={'User-Agent': ''})
soup = BeautifulSoup(req.content, 'html.parser')

In [14]:
urls = []
for link in soup.find_all('loc'):
    urls.append(link.string)

In [15]:
for i in tqdm_notebook(range(len(urls))):
    printError(urls[i])

  0%|          | 0/428 [00:00<?, ?it/s]

https://studentlife.umich.edu/parents/
	Wednesday 2/16 4:30-5:30pm
	https://umich.zoom.us/webinar/register/WN_gDFMeLfvTAmVxHDibFJ41w

	Friday 2/18 12:00-1:00pm
	https://umich.zoom.us/webinar/register/WN_u1hczk97SNGiAArfl6PI7Q

	Friday 2/18 4:30-5:30pm
	https://umich.zoom.us/webinar/register/WN_1q2LGnpaTqqho9oVXDRqdg

	Monday 2/21, 6:30-7:30pm
	https://umich.zoom.us/webinar/register/WN_wceHNeqZSIGMObOIWTbWUQ

	Tuesday 2/22, 5:30-6:30pm
	https://umich.zoom.us/webinar/register/WN_7EcbjTO1SoWFL7SPS7EJLA

	Wednesday 2/23 4:30-5:30pm
	https://umich.zoom.us/webinar/register/WN_P3aNEkhQTyGnr8BXoItMSg


https://studentlife.umich.edu/parents/article/services-students-disabilities-fact-fund
	needed technology
	http://ssd.umich.edu/article/what-assistive-technology

	http://ssd.umich.edu/article/giving
	http://ssd.umich.edu/article/giving


https://studentlife.umich.edu/parents/article/campus-safety-community-effort
	bus transportation
	http://pts.umich.edu/taking_the_bus/index.php

	Emergency Ale